In [3]:
import pandas as pd
import numpy as np
from __future__ import print_function
from __future__ import division
from __future__ import unicode_literals

import numpy as np
import deepchem as dc
from deepchem.molnet import load_delaney
from deepchem.models import GraphConvModel
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [54]:
from rdkit import Chem
from rdkit.Chem import Descriptors
m = Chem.MolFromSmiles('C1CC(CCn2cncn2)CN(C1)c1snc2ccccc12')

In [89]:
list_of_Descriptors = dir(Descriptors)
# for f in list_of_Descriptors:
#     print (Descriptors.f(m))
# Descriptors.list_of_Descriptors[0]
# for f in list_of_Descriptors:
#     print("Descriptors."+f+"(m),")
Descriptors.fr_urea(m)

0

In [91]:
list_jinga = [Descriptors.BalabanJ(m),
Descriptors.BertzCT(m),
Descriptors.Chi0(m),
Descriptors.Chi0n(m),
Descriptors.Chi0v(m),
Descriptors.Chi1(m),
Descriptors.Chi1n(m),
Descriptors.Chi1v(m),
Descriptors.Chi2n(m),
Descriptors.Chi2v(m),
Descriptors.Chi3n(m),
Descriptors.Chi3v(m),
Descriptors.Chi4n(m),
Descriptors.Chi4v(m),
Descriptors.EState_VSA1(m),
Descriptors.EState_VSA10(m),
Descriptors.EState_VSA11(m),
Descriptors.EState_VSA2(m),
Descriptors.EState_VSA3(m),
Descriptors.EState_VSA4(m),
Descriptors.EState_VSA5(m),
Descriptors.EState_VSA6(m),
Descriptors.EState_VSA7(m),
Descriptors.EState_VSA8(m),
Descriptors.EState_VSA9(m),
Descriptors.ExactMolWt(m),
Descriptors.FpDensityMorgan1(m),
Descriptors.FpDensityMorgan2(m),
Descriptors.FpDensityMorgan3(m),
Descriptors.FractionCSP3(m),
Descriptors.HallKierAlpha(m),
Descriptors.HeavyAtomCount(m),
Descriptors.HeavyAtomMolWt(m),
Descriptors.Ipc(m),
Descriptors.Kappa1(m),
Descriptors.Kappa2(m),
Descriptors.Kappa3(m),
Descriptors.LabuteASA(m),
Descriptors.MaxAbsEStateIndex(m),
Descriptors.MaxAbsPartialCharge(m),
Descriptors.MaxEStateIndex(m),
Descriptors.MaxPartialCharge(m),
Descriptors.MinAbsEStateIndex(m),
Descriptors.MinAbsPartialCharge(m),
Descriptors.MinEStateIndex(m),
Descriptors.MinPartialCharge(m),
Descriptors.MolLogP(m),
Descriptors.MolMR(m),
Descriptors.MolWt(m),
Descriptors.NHOHCount(m),
Descriptors.NOCount(m),
Descriptors.NumAliphaticCarbocycles(m),
Descriptors.NumAliphaticHeterocycles(m),
Descriptors.NumAliphaticRings(m),
Descriptors.NumAromaticCarbocycles(m),
Descriptors.NumAromaticHeterocycles(m),
Descriptors.NumAromaticRings(m),
Descriptors.NumHAcceptors(m),
Descriptors.NumHDonors(m),
Descriptors.NumHeteroatoms(m),
Descriptors.NumRadicalElectrons(m),
Descriptors.NumRotatableBonds(m),
Descriptors.NumSaturatedCarbocycles(m),
Descriptors.NumSaturatedHeterocycles(m),
Descriptors.NumSaturatedRings(m),
Descriptors.NumValenceElectrons(m),
Descriptors.PEOE_VSA1(m),
Descriptors.PEOE_VSA10(m),
Descriptors.PEOE_VSA11(m),
Descriptors.PEOE_VSA12(m),
Descriptors.PEOE_VSA13(m),
Descriptors.PEOE_VSA14(m),
Descriptors.PEOE_VSA2(m),
Descriptors.PEOE_VSA3(m),
Descriptors.PEOE_VSA4(m),
Descriptors.PEOE_VSA5(m),
Descriptors.PEOE_VSA6(m),
Descriptors.PEOE_VSA7(m),
Descriptors.PEOE_VSA8(m),
Descriptors.PEOE_VSA9(m),
Descriptors.RingCount(m),
Descriptors.SMR_VSA1(m),
Descriptors.SMR_VSA10(m),
Descriptors.SMR_VSA2(m),
Descriptors.SMR_VSA3(m),
Descriptors.SMR_VSA4(m),
Descriptors.SMR_VSA5(m),
Descriptors.SMR_VSA6(m),
Descriptors.SMR_VSA7(m),
Descriptors.SMR_VSA8(m),
Descriptors.SMR_VSA9(m),
Descriptors.SlogP_VSA1(m),
Descriptors.SlogP_VSA10(m),
Descriptors.SlogP_VSA11(m),
Descriptors.SlogP_VSA12(m),
Descriptors.SlogP_VSA2(m),
Descriptors.SlogP_VSA3(m),
Descriptors.SlogP_VSA4(m),
Descriptors.SlogP_VSA5(m),
Descriptors.SlogP_VSA6(m),
Descriptors.SlogP_VSA7(m),
Descriptors.SlogP_VSA8(m),
Descriptors.SlogP_VSA9(m),
Descriptors.TPSA(m),
Descriptors.VSA_EState1(m),
Descriptors.VSA_EState10(m),
Descriptors.VSA_EState2(m),
Descriptors.VSA_EState3(m),
Descriptors.VSA_EState4(m),
Descriptors.VSA_EState5(m),
Descriptors.VSA_EState6(m),
Descriptors.VSA_EState7(m),
Descriptors.VSA_EState8(m),
Descriptors.VSA_EState9(m),
Descriptors.fr_Al_COO(m),
Descriptors.fr_Al_OH(m),
Descriptors.fr_Al_OH_noTert(m),
Descriptors.fr_ArN(m),
Descriptors.fr_Ar_COO(m),
Descriptors.fr_Ar_N(m),
Descriptors.fr_Ar_NH(m),
Descriptors.fr_Ar_OH(m),
Descriptors.fr_COO(m),
Descriptors.fr_COO2(m),
Descriptors.fr_C_O(m),
Descriptors.fr_C_O_noCOO(m),
Descriptors.fr_C_S(m),
Descriptors.fr_HOCCN(m),
Descriptors.fr_Imine(m),
Descriptors.fr_NH0(m),
Descriptors.fr_NH1(m),
Descriptors.fr_NH2(m),
Descriptors.fr_N_O(m),
Descriptors.fr_Ndealkylation1(m),
Descriptors.fr_Ndealkylation2(m),
Descriptors.fr_Nhpyrrole(m),
Descriptors.fr_SH(m),
Descriptors.fr_aldehyde(m),
Descriptors.fr_alkyl_carbamate(m),
Descriptors.fr_alkyl_halide(m),
Descriptors.fr_allylic_oxid(m),
Descriptors.fr_amide(m),
Descriptors.fr_amidine(m),
Descriptors.fr_aniline(m),
Descriptors.fr_aryl_methyl(m),
Descriptors.fr_azide(m),
Descriptors.fr_azo(m),
Descriptors.fr_barbitur(m),
Descriptors.fr_benzene(m),
Descriptors.fr_benzodiazepine(m),
Descriptors.fr_bicyclic(m),
Descriptors.fr_diazo(m),
Descriptors.fr_dihydropyridine(m),
Descriptors.fr_epoxide(m),
Descriptors.fr_ester(m),
Descriptors.fr_ether(m),
Descriptors.fr_furan(m),
Descriptors.fr_guanido(m),
Descriptors.fr_halogen(m),
Descriptors.fr_hdrzine(m),
Descriptors.fr_hdrzone(m),
Descriptors.fr_imidazole(m),
Descriptors.fr_imide(m),
Descriptors.fr_isocyan(m),
Descriptors.fr_isothiocyan(m),
Descriptors.fr_ketone(m),
Descriptors.fr_ketone_Topliss(m),
Descriptors.fr_lactam(m),
Descriptors.fr_lactone(m),
Descriptors.fr_methoxy(m),
Descriptors.fr_morpholine(m),
Descriptors.fr_nitrile(m),
Descriptors.fr_nitro(m),
Descriptors.fr_nitro_arom(m),
Descriptors.fr_nitro_arom_nonortho(m),
Descriptors.fr_nitroso(m),
Descriptors.fr_oxazole(m),
Descriptors.fr_oxime(m),
Descriptors.fr_para_hydroxylation(m),
Descriptors.fr_phenol(m),
Descriptors.fr_phenol_noOrthoHbond(m),
Descriptors.fr_phos_acid(m),
Descriptors.fr_phos_ester(m),
Descriptors.fr_piperdine(m),
Descriptors.fr_piperzine(m),
Descriptors.fr_priamide(m),
Descriptors.fr_prisulfonamd(m),
Descriptors.fr_pyridine(m),
Descriptors.fr_quatN(m),
Descriptors.fr_sulfide(m),
Descriptors.fr_sulfonamd(m),
Descriptors.fr_sulfone(m),
Descriptors.fr_term_acetylene(m),
Descriptors.fr_tetrazole(m),
Descriptors.fr_thiazole(m),
Descriptors.fr_thiocyan(m),
Descriptors.fr_thiophene(m),
Descriptors.fr_unbrch_alkane(m),
Descriptors.fr_urea(m)
]

In [92]:
list_jinga

[1.4608985438263866,
 735.9267292578912,
 14.77781011412252,
 12.428408839410322,
 13.244905420338048,
 10.898979485566358,
 7.817485924187317,
 8.590882586321289,
 5.906374173270629,
 6.658220876007727,
 4.404146866623924,
 5.3431791513691085,
 3.251946776826872,
 4.148220975968816,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 37.48969760444139,
 23.228949437004683,
 24.187126761095758,
 4.681802935145185,
 43.62157831536161,
 0.0,
 313.136116608,
 1.2727272727272727,
 2.227272727272727,
 3.1363636363636362,
 0.4375,
 -1.9499999999999993,
 22,
 294.2780000000001,
 308169.8591980298,
 13.695013904508262,
 6.042175350040524,
 2.7041709774131917,
 133.30674185295743,
 4.583142501988552,
 0.3614028720176149,
 4.583142501988552,
 0.13725250105158748,
 0.7211224489795917,
 0.13725250105158748,
 0.7211224489795917,
 -0.3614028720176149,
 3.1945000000000014,
 89.19600000000004,
 313.43,
 0,
 5,
 0,
 1,
 1,
 1,
 2,
 3,
 6,
 0,
 6,
 0,
 4,
 0,
 1,
 1,
 114,
 4.899909730850478,
 17.65572212621695,
 0.0,
 0.0,

['X',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_construct_metadata',
 '_get_metadata_filename',
 'add_shard',
 'complete_shuffle',
 'create_dataset',
 'data_dir',
 'from_numpy',
 'get_data_shape',
 'get_label_means',
 'get_label_stds',
 'get_number_shards',
 'get_shape',
 'get_shard',
 'get_shard_size',
 'get_statistics',
 'get_task_names',
 'ids',
 'iterbatches',
 'itersamples',
 'itershards',
 'load_metadata',
 'make_iterator',
 'merge',
 'metadata_df',
 'move',
 'reshard',
 'save_to_disk',
 'select',
 'set_shard',
 'shuffle_each_shard',
 'shuffle_shards',
 'sparse_shuffle',
 'subset',
 'tasks',
 'transform',
 'verbose',
 'w',
 'write_data_to_disk

In [24]:
dataset.write_data_to_disk(data_dir="/Users/KHatti001/OneDrive - University of Dundee/project1_data",
                          basename = "jinga",
                          tasks = 'CL mouse GEOM_MEAN')

[None, None, None, None]